# DNDS6013 Scientific Python: 12th Class
## Central European University, Winter 2019/2020

Instructor: Márton Pósfai, TA: Luis Natera Orozco

Emails: posfaim@ceu.edu, natera_luis@phd.ceu.edu



To complete the class, go though this notebook and
* Study the example codes
* Solve the exercises in the notebook. Try looking at solutions only when you are done.
* Follow links to videos for short verbal explanations if needed
* Complete a final task and upload your result to Moodle, pay attention to upload only **a single pdf figure, do not upload your code**.

If you have any questions or you get stuck with one of the exercises I will be available on the [slack channel](http://sp2020winter.slack.com). I will be online during regular class hours, outside of that I will try to get back to you as soon as possible.




## Today's plan:

We will continue our introduction to [pandas](http://pandas.pydata.org/):
- Use pandas to analyze a bike sharing system
- Use the datetime functionality of pandas
- Do more plotting and take a brief look at the [seaborn](https://seaborn.pydata.org/) library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.__version__

## First: when to use which module?

[Video](https://ceu.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=fc70308b-26c8-4b0b-8fd5-ab8b00d12224)

During this term, we learned about many python modules some of the have similar functionality, and we have seen that any given problem can be solved in multiple ways and with multiple tools. For example, to store data python has built in data structures like lists and dictionaries, numpy as n-dimensional arrays, and right now we are learning about pandas' series and dataframe, all of them have overlapping usage.

So how to choose the best tool for your problem? There is of course no strict rule, but the main guidline that I recommend following is: use the simplest tool that allows you to concisely complete your task. Let's look at the question which datastructure to use:

* **Lists, dictionaries:** These are our most simple options, use it for small common task such as passing parameters to a function, or when you are not planning to use special features of more complex modules. They are built in datatypes of python; therefore they don't introduce additional dependency and your code will be more portable.
* **numpy arrays:** Numpy arrays are built for performance, they are the most useful when you have purely numeric data and you do intensive calculations with them. Applications include, but are not limited to, optimization problems, solving equations numerically, image processing, etc. Many additional modules rely on numpy, including pandas.
* **pandas:** Use them if your data is spreadsheet-like, requires cleaning or merging. Also pandas makes many common tasks very easy, such as dealing with timestamps or simple data exploration and visualization (and other things we cover in class).


# Part III: Bikesharing with Pandas

[Video](https://ceu.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=fd4f3a1a-5b98-4a17-be01-ab8b00d637a2)

Let's look at bike sharing data from Chicago. In Chicago there is a bike-sharing program just like Budapest's Bubi, called Divvy. We have two datasets (retrieved from https://www.divvybikes.com/system-data): one on trips and another on stations. We'll have to combine the two to explore how people use Divvy.

In [ ]:
trips = pd.read_csv('Divvy_Trips_2013.csv')

How many trips does our data set contain? In other words, how many rows are there in the dataframe?

In [ ]:
len(trips)

Let's take a look at what we have:

In [ ]:
trips.head()

Basic summary of data frame: 

In [ ]:
trips.info()

## Column meanings:
- trip_id: trip identifier
- starttime: time bike was rented
- stoptime: time bike was returned
- bikeid: bike identifier
- trip duration: duration of the trip in seconds
- from_station_id: station id from which bike was borrowed
- from_station_name: station name from which bike was borrowed
- to_station_id: station id to which bike was returned
- to_station_name: sstation id to which bike was returned
- usertype: Customer or subscribers. Customers are one time users that pay for single trips, subscribers buy a monthly -pass with unlimited rides.
- gender: Gender, if known, of user
- birthyear: year of birth, if known, of user

We are going to continue with exploring the data.

### Heterogeneous trip durations

[Video](https://ceu.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=734d14b0-992c-41e9-bc16-ab8b00d76ffc)

For example, let's plot the trip duration distribution.

In [ ]:
trips['tripduration'].plot(kind='hist',bins=50);

This is not too helpful, we have a few trips that are much longer than average. 
Let's create a new column with the logarithm of the tripduration and plot that.

We can use the `apply()`: 

In [ ]:
trips['logduration']=trips['tripduration'].apply(np.log)

Or since `np.log()` is a numpy function, we can simply write:

In [ ]:
trips['logduration']=np.log(trips['tripduration'])

In [ ]:
ax=trips['logduration'].plot(kind='hist',bins=50,color='#9BCC31')
ax.set_xlabel('Log(trip duration)');

This looks better. What this means is that tripduration has a very wide distribution, which is closer to a lognormal than a normal.

### Most used bikes

Let's see which bike is on the road the longest: perhaps a good candidate for early retirement. The `bikeid` column uniquely identifies the bikes, to find all trips associated to each bike we can use the `groupby()` function:

In [ ]:
bike_usage = trips.groupby('bikeid')['tripduration'].sum().sort_values(ascending=False)

Let's unpack this line of code:
1. `groupby(bikeid)`: this creates a group for each individual bike
2. `['tripduration']`: pick the `tripduration` column
3. `sum()`: sum all trips for each bike, so we get a series containing the total usage and indexed by the bike IDs
4. `sort_values(ascending=False)`: sort in descending order so that the most used bike is in the first row

The most used bikes are in the first rows, we can print out the top 10 using `head()`

In [ ]:
bike_usage.head(10)

The least used bikes are at the end:

In [ ]:
bike_usage.tail(10)

We can also plot the distribution:

In [ ]:
ax = bike_usage.hist(bins=40,color='g');
ax.set_ylabel("count")
ax.set_xlabel("total usage");

### Exercise

Repeat the previous analysis, but instead of the total duration of usage, look at the total number of times the bikes were used. Do you get the same bikes in the top 10?

<details><summary><u>Hint</u></summary>
<p>
    
The only difference is that instead of using the `sum()` function to aggregate the data for each bike, we use the `count()` function, which simply counts the number of elements in the group. 

</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
bike_usage2 = trips.groupby('bikeid')['tripduration'].count().sort_values(ascending=False)
ax = bike_usage2.hist(bins=40,color='g');
ax.set_ylabel("count")
ax.set_xlabel("total number of times used");
```
    
</p>
</details>

### Timestamps

[Video](https://ceu.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=bc18a9de-d075-4d9f-9f7c-ab8b00d9ee84)

We can use the timestamps associated to each trip to investigate such questions as how usage depends on the time of the year, day of the week or part of the day.

How can we make use of these timestamps? First, let's see how the timestamps are currently stored.

In [ ]:
print(trips['starttime'][0])
print(type(trips['starttime'][0]))

Let's turn those time columns into pandas' timestamp objects, which are similar to python's datetime objects. It is a common task so pandas has a built in function for it:

In [ ]:
trips.starttime = pd.to_datetime(trips.starttime, format="%Y-%m-%d %H:%M")
trips.stoptime = pd.to_datetime(trips.stoptime, format="%Y-%m-%d %H:%M")

print(trips['starttime'][0])
print(type(trips['starttime'][0]))

(Side note: we can access a column by name two ways, `trips['starttime']` is the same as `trips.starttime`.)

Note about `to_datetime()`: if you don't pass a format pandas will try to guess it. It does a decent job, but takes slower and is risky. One can also pass `errors = "coerce"` and it will return `NaT` (Not a Time) values for those entries that don't fit into the format you give.

Now we can use these columns to plot stuff. For example, let's see the variation in duration by start time across the year. Plotting as a function of time has never been easier:

In [ ]:
ax = trips.plot(kind='line', x='starttime', y='tripduration', figsize=(14,5));
#trips.rolling('5D').mean().plot(kind='line', x='starttime', y='tripduration',ax=ax);

This is a mess, let's try to instead look at daily averages. Currently the timestamps have minute precision, let's create a column that only contains the day. For this, we can use the `date` method of timestamp objects. As a demonstration let's apply it to the first timestamp in the dataset:

In [ ]:
trips['starttime'][0].date()

We create a new column using `apply()`:

In [ ]:
trips['startdate']=trips['starttime'].apply(lambda dt: dt.date())

Now we group by the date and calculate the average:

In [ ]:
daily_avg_tripduration = trips.groupby('startdate')['tripduration'].mean()

This now a series that is indexed by the date and contains the average trip duration for each day:

In [ ]:
daily_avg_tripduration.head()

Let's plot this:

In [ ]:
daily_avg_tripduration.plot(kind='line',figsize=(14,5));

We see two patterns:
* There is an overall decreasing trend: people go on shorter trips in the winter
* Usage is periodic
* Bonus question: can you guess why might we see that peak on the first day? (Hint: think of the shape of the distribution of the trip duration and sample sizes.)

Let's investigate the origin of the periodicity! Perhaps it is a weekly pattern? 

### Exercise

Repeat the previous analysis, but instead of doing a daily average for each day, calculate the average for the days of the week, i.e., you will get seven datapoints, the average trip duration for each day of the week.
* Create a new column named `'dayofweek'` that contains the day of the week the trip started
* Group the trips based on this new column
* Plot the result

What did you find?

<details><summary><u>Hint</u></summary>
<p>

To get the day of the week using the `dayofweek` attribute of timestamps. Pay attention: this is not a function, there is no parenthesis at the end. For example, the day of week of the first timestamp is `trips.['starttime'][0].weekofday`.

</p>
</details>

In [ ]:
trips['dayofweek']=trips['starttime'].apply(lambda dt: dt.dayofweek)
dayofweek_avg_tripduration = trips.groupby('dayofweek')['tripduration'].mean()
dayofweek_avg_tripduration.plot(kind='line',figsize=(10,5));

<details><summary><u>Solution.</u></summary>
<p>
    
```python
trips['dayofweek']=trips['starttime'].apply(lambda dt: dt.dayofweek)
dayofweek_avg_tripduration = trips.groupby('dayofweek')['tripduration'].mean()
dayofweek_avg_tripduration.plot(kind='line',figsize=(10,5));
```
    
</p>
</details>

### Plotting with Seaborn

Pandas plotting is simple but limited, matplotlib plotting is complicated but powerful. A library called [seaborn](https://seaborn.pydata.org/) is in the middle and it works very well with data in pandas. We will very briefly look at few things that we can do with it.

In [ ]:
import seaborn as sns
sns.set_style('white')# you can try other styles

Do women and men use Divvy the same way? Let's look at the previous plot, but separate it for the two groups. We can use `groupby()` to group according to two columns by passing a list containing the names of the column to `groupby()` (we did something similar with the Titanic dataset)

In [ ]:
dayofweek_gender_duration = trips.groupby(['dayofweek','gender'])['tripduration'].mean()

We get a series with hierarchical indices (`dayofweek` and `gender`):

In [ ]:
dayofweek_gender_duration.head()

For plotting we will need a dataframe. We can use `reset_index()` to convert the indices to regular columns and index the rows with integers instead:

In [ ]:
dayofweek_gender_duration = dayofweek_gender_duration.reset_index()
dayofweek_gender_duration.head()

As you can see, this is now a dataframe which is indexed by integers, and the previous indices became regular columns `dayofweek` and `gender`.

We use seaborn to create a more fancy plot: 

In [ ]:
ax=sns.pointplot(data=dayofweek_gender_duration,x='dayofweek',y='tripduration', hue='gender')
ax.set_ylabel('Average duration of trips')
ax.set_title('Average trip duration per day user gender');
# Seaborn has a bunch of formatting options
# e.g., despine removes top and right sides of the frame
sns.despine()

It seems that men go on shorter duration trips, but their trip length increases more on weekends.

### Exercise

Our dataset also has a column named `usertype`, it differentiates between customer or subscribers. Customers are one time users that pay for single trips, subscribers buy a monthly -pass with unlimited rides.

Do customers and subscribers behave differently? Repeat the above analysis for these categories. In addition to trip duration, compare total number of trips as well in a separate cell. What pattern do you see?

<details><summary><u>Hint</u></summary>
<p>

You can do exatly the same as before only:
* group by `usertype` instead of `gender`
* also repeat your plot for `count()` instead of `mean()`

</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
dayofweek_usertype_duration = trips.groupby(['dayofweek','usertype'])['tripduration'].mean().reset_index()
ax=sns.pointplot(data=dayofweek_usertype_duration,x='dayofweek',y='tripduration', hue='usertype')
ax.set_ylabel('Average duration of trips')
ax.set_title('Average trip duration per day user type');
sns.despine()

dayofweek_usertype_duration = trips.groupby(['dayofweek','usertype'])['tripduration'].count().reset_index()
ax=sns.pointplot(data=dayofweek_usertype_duration,x='dayofweek',y='tripduration', hue='usertype')
ax.set_ylabel('Number of trips')
ax.set_title('Number of trips per user type');
sns.despine()
```
    
</p>
</details>

We don't have to restrict ourselves to looking at the only at the daily average. Seaborn has a convinient way to plot distributions for different categories. Let's plot the logarithim of the tripduration for each day and the two genders.

(To show the histograms, we are going to use the [kernel density estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation) plot of seaborn, which is basically a smoothed version of the raw histogram.)

In [ ]:
g = sns.FacetGrid(trips, col='dayofweek', hue='gender', col_wrap=4) #specify the grid
g.map(sns.kdeplot, 'logduration') #specify the type of plot, here: kernel density estimate
g.add_legend(); #add legend

Seaborn can handle many types of plots that we don't have time to cover in the class. [Click here to explore](https://seaborn.pydata.org/examples/index.html).

### Adding station data

Recall that we also have information about stations in 'Divvy_Stations_2013.csv'. Let's have a look at this data and add it to our trips dataframe.

In [ ]:
stations = pd.read_csv('Divvy_Stations_2013.csv')

In [ ]:
stations[['name','latitude','longitude']].loc[(stations['name'] == "Michigan Ave & Oak St") | (stations['name'] == "Racine Ave & Congress Pkwy")| (stations['name'] == "Loomis St & Taylor St")].head(3)

In [ ]:
trips[['trip_id',"starttime","from_station_name","to_station_name"]].head(3)

In [ ]:
print(len(stations))
stations.head()

So we have a much smaller dataframe with totally different rows. We want to join the stations dataframe to the trips dataframe so that we have lat/long/capacity data for each trips' starting and ending stations. The name column in the stations frame corresponds to the `from_station_name` and `to_station_name columns` in the trip frame. For this task we can use the [merge()](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html) function.

[Video](https://ceu.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=b3628637-bd80-47f6-8225-ab8b00dd9d62)

First, merge trips and stations for the starting stations. Let's unpack the following function call:
* `merge()` takes two dataframes `left` and `right`
* `how='left'` means that we take the `left` dataframe, in this case `trips`, and we add new columns to it based on the `right` dataframe, in this case `stations`. Therefore in the new dataframe we will have the same number of rows as in `trips`.
* `left_on='from_station_name'` and `right_on='name'`: `merge()` takes a rew in `left` looks at the `from_station_name` and looks for a match in the `right` dataframe.

In [ ]:
trips2 = pd.merge(left=trips, right=stations, how='left', left_on='from_station_name', right_on='name')

trips2[['trip_id',"starttime","from_station_name","to_station_name",'latitude','longitude']].head(3)

Now we have a dataframe that contains the trips + information about the starting station. For example, the `lattitude` column contains the lattitude of the starting station.

We also want to add the information about the destination stations. If we do the same as before, we would get duplicate column names, e.g., two `lattitude` columns. To avoid this `merge()` can add a suffix to the column names:

In [ ]:
#added a suffix list for duplicated names
trips_extended = pd.merge(trips2, stations, how='inner', left_on='to_station_name', right_on='name',
                    suffixes=['_origin', '_dest'])


We check the first three rows of the new dataframe:

In [ ]:
trips_extended.head(3)

Now we have a bunch of new data to work with!

### Exercise

What are the most popular routes? Create a dataframe called `station_to_station` that has a row for each station origin-destination pair and has columns:
* `from_station_name`
* `to_station_name`
* And a column containing the total number of trips between the two stations. (Bonus: rename this column `numtrips`.)
* Print out the top 10 station pairs

<details><summary><u>Hint 1.</u></summary>
<p>

Group `trips_extended` by two columns: `'from_station_name'` and `'to_station_name'`.

</p>
</details>

<details><summary><u>Hint 2.</u></summary>
<p>

To aggregate the data for each group use `count()`.

</p>
</details>


<details><summary><u>Hint 3.</u></summary>
<p>

You can rename columns using `df.rename(columns = {'old_col_name':'new_col_name'},inplace=True)`. For details look up the documentation!

</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
station_to_station = trips_extended.groupby(['from_station_name','to_station_name'])['trip_id'].count().reset_index()
station_to_station.rename(columns = {'trip_id':'numtrips'},inplace=True)
station_to_station.sort_values('numtrips', ascending=False).head()
```
    
</p>
</details>

### Network of stations

We don't have to restrict ourselves to analyzing the data with pandas. The `station_to_station` dataframe can be thought of as a weighted directed edge list. Let's build a network out of it and plot it!

How many nodes and edges are there?

In [ ]:
print("Number of nodes:", len(stations))
print("Number of edges:", len(station_to_station))
print("Maximum number of possible directed edges:", len(stations)*len(stations))

Almost half of all possible edges are present, this is too dense for normal network analysis. Let's threshold the edges and only include the most possible routes, i.e., the station pairs that have more than `T` trips.

Let's filter the `station_to_station` dataframe:

In [ ]:
T = 150

filtered_station_to_station = station_to_station[station_to_station['numtrips']>T]
print(len(filtered_station_to_station))

Now let's create a networkx network.

In [ ]:
import networkx as nx

g = nx.DiGraph()


We can add edges using `g.add_edges_from(edgelist)`, where `edgelist` is an iterable containing `(src,dest)` tuples representing the edges. We can create a series like this using `apply()`:

In [ ]:
edgelist = filtered_station_to_station.apply(lambda row: (row['from_station_name'],row['to_station_name']),axis=1)
edgelist.head()

g.add_edges_from(edgelist)

Now, let's plot this network:

In [ ]:
pos = nx.kamada_kawai_layout(g)
plt.figure(figsize=(10,5))
nx.draw_networkx_nodes(g, pos, node_size=50, node_color="#00aaaa") #draw nodes
nx.draw_networkx_edges(g, pos, edgelist=g.edges()) #draw edges

plt.axis('off');

We stop here, but the interested reader could further analyse the network, e.g., look at different centralites or community structure.

## Additional exercises

### Hours of day

Using the `hour` attribute of the timestamp objects, create a new column that contains the hour of the day when the trip started. Plot the average trip duration and/or total number of trips that happened in each hour with a separate color for each day of the week using seaborn.

Can you explain the patterns that you see?

<details><summary><u>Hint 1</u></summary>
<p>

Creating the new column: This is very similar to what we did with the day of week, only instead of using `dt.dayofweek`, we have to use `dt.hour`.

</p>
</details>

<details><summary><u>Hint 2</u></summary>
<p>

Plotting: This is very similar to what we did with the `dayofweek` and `gender`, only instead of using `gender` use the new column. Make sure that the hue represents the days of the week and the x axis represents the hour of the day.

</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
trips['hour']=trips['starttime'].apply(lambda dt: dt.hour)

dayofweek_usertype_duration = trips.groupby(['dayofweek','hour'])['tripduration'].count().reset_index()
ax=sns.pointplot(data=dayofweek_usertype_duration,x='hour',y='tripduration', hue='dayofweek')
ax.set_ylabel('Number of trips')
ax.set_title('Number of trips per user type');
sns.despine()
```
    
</p>
</details>

### Capacity

Investigate the relationship between station capacity and traffic. Which two stations would you expand if you had the budget?
* Create a series indexed by the station name and containing the total number of out-going traffic of each station
* Do the same thing with out-traffic and add the two series together to get the total traffic
* Use `merge()` to add traffic data to the `stations` dataframe
* Create a scatter plot showing the correlation between traffic and capacity

<details><summary><u>Hint 1</u></summary>
<p>

To get the out-traffic group the trips by `from_station_name` and use `count()` on the `trip_id` column.

</p>
</details>

<details><summary><u>Hint 2</u></summary>
<p>

Use `merge()` very similarly as we did earlier to extend the `trips` dataframe. Try renaming the new column!
The next hint reveals the exact code to do the merge.

</p>
</details>

<details><summary><u>Hint 3</u></summary>
<p>

```python
stations_extended = pd.merge(left=stations, right=traffic, how='left', left_on='name', right_on='to_station_name')
```
The series object has a name attribute, in this case `traffic.name`, that will become the name of the column after the merge. You can rename the column after the merge, or you can change the name of the series before the merge.

</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
outtraffic = trips_extended.groupby('from_station_name')['trip_id'].count()
intraffic = trips_extended.groupby('to_station_name')['trip_id'].count()
traffic = intraffic+outtraffic
traffic.name = "traffic"
stations_extended = pd.merge(left=stations, right=traffic, how='left', left_on='name', right_on='to_station_name')
stations_extended.plot(kind='scatter',x='dpcapacity',y='traffic');
```
    
</p>
</details>

### Sources and sinks

Which stations are sources and which stations are sinks, i.e. which stations have much more departures than arrivals, and vice versa? Print out the top 10 stations with the largest in- and out-traffic difference.

<details><summary><u>Hint.</u></summary>
<p>

Use the part of the solution of the previous exercise.

To sort a series `S` by its values use `S.sort_values()`.
</p>
</details>

<details><summary><u>Solution.</u></summary>
<p>
    
```python
traffic_diff = np.abs(intraffic-outtraffic).sort_values(ascending=False)
traffic_diff.head(10)
```
    
</p>
</details>

## Final exercise

Pick **one** of the following problems and create a figure. Upload only a **single figure as a pdf** to Moodle, do not upload your code. Successfully completing this task counts as attendance.



### 1. Distances

Let's pull up our old favorite, the [haversine formula](https://en.wikipedia.org/wiki/Haversine_formula)! It converts lattitude and longitude to distance (in this case, kilometers):

In [ ]:
import math
def latlongdist(lat1,long1,lat2,long2):
    rlat1 = math.radians(lat1)
    rlat2 = math.radians(lat2)
    rlong1 = math.radians(long1)
    rlong2 = math.radians(long2)
    dlat = rlat2 - rlat1
    dlong = rlong2 - rlong1
    a = math.sin(dlat / 2)**2 + math.cos(rlat1) * math.cos(rlat2) * math.sin(dlong / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6371.0 * c

print(latlongdist(48.105625, 20.790556, 46.07308, 18.22857))

Use this formula to add a new column to the `trips_extended` dataframe that contains the distance (as the crow flies) between the origin and destination stations. Then create a figure to answer one of the following questions:
* Does the trip duration and distance correlate?
* Do men or women go on longer trips?
* How does the trip distance depend on the day of the week?

### 2. Age

Using the `birthday` column in `trips` calculate the age of the rider in years and create a new column `age`. Then answer one of these questions:
* What is the age distribution of riders depending on their gender?
* Does the trip duration depend on age?

### 3. Other

Come up with an interesting plot using this data set.